In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
# Считывание данных

sot = pd.read_csv('sotrudniki_hakaton_utf.csv', sep = ';')
rod = pd.read_csv('rodstvenniki_hakaton_utf_v2.csv', sep = ';')
ogrv = pd.read_csv('OGRV_hakaton_utf.csv', sep = ';')

In [3]:
# Создание вспомогательного датафрейма с информацией о количестве смен сотрудника в месяце

ogrv['month'] = ogrv['date'].map(lambda x: x[0:8] + str('01'))
kolvo_smen = ogrv[ogrv.work_shift_type.isin(['Смена 1', 'Смена 2', 'Смена 3'])]\
[['hash_tab_num','month','work_shift_type']].groupby(['hash_tab_num','month']).agg('count').reset_index()
kolvo_smen.columns = ['hash_tab_num', 'date', 'work_shift_type_count']

In [4]:
# Создание вспомогательного датафрейма с информацией о факте больничного в текущем месяце

kolvo_bolni4 = ogrv[ogrv.graphic_rule_level_1.isin(['Больничный'])]\
[['hash_tab_num','month','graphic_rule_level_1']].groupby(['hash_tab_num','month']).agg('count').reset_index()

kolvo_bolni4['graphic_rule_level_1'] = 1
kolvo_bolni4.columns = ['hash_tab_num', 'date', 'sick']

In [5]:
# Базовый датафремй

sot_data = sot[['hash_tab_num','date','category','gender','razryad_fact','work_experience_company',
                'name_fact_lvl5','education','home_to_work_distance']]
sot_data.gender = sot_data['gender'].map(lambda x: 1 if x == 'мужской' else 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
# Создание вспомогательно датасета с информацией о родственниках - пенсионерах
# (55 лет для женщин и 60 лет для мужчин для региона севера)

sot_data = pd.merge(sot_data,rod, how = 'left', on = 'hash_tab_num')
sot_data['rel_cur_old'] = ([int(x[0:4]) for x in sot_data['date']] - sot_data['rel_birth'])
sot_data['rel_is_male'] = sot_data.rel_type.map(lambda x:1 if x \
    in ['Сын', 'Муж', 'Отец', 'Пасынок', 'Внук','Брат'] else 0)

retiree = sot_data[((sot_data.rel_cur_old > 55) & (sot_data.rel_is_male == 0) \
               | (sot_data.rel_cur_old > 60) & (sot_data.rel_is_male == 1))]\
    [['hash_tab_num','date','rel_is_male']].groupby(['hash_tab_num','date']).agg('count').reset_index()
sot_data.drop(['rel_type','rel_birth','rel_cur_old','rel_is_male'], axis = 1, inplace = True)

In [7]:
# Создание вспомогательно датасета с информацией о количестве сотрудников в подразделении
# по фактическому месту работы

division_count = sot_data[['hash_tab_num','date','name_fact_lvl5']].\
groupby(['name_fact_lvl5','date']).agg('count').reset_index()

division_count.columns = ['name_fact_lvl5', 'date', 'personel_num']

sot_data = pd.merge(sot_data, division_count, how = 'left', on = ['date','name_fact_lvl5'])

In [8]:
# Создание dummy переменных

sot_data.education = sot_data['education']\
.map(lambda x: 'Высшее' if x in ['Высшее образование','Высшее-бакалавриат','Высшее-специалитет'] else(\
'Среднее_профессинальное' if x in ['Ср.профессиональное','Нач.профессиональное'] else 'Начальное_среднее'))
sot_data = pd.get_dummies(sot_data, columns = ['category','education','razryad_fact'])\
.drop('name_fact_lvl5', axis = 1)

In [9]:
# Создание единого датасета для будущего использования в модели

merged_data = pd.merge(sot_data, retiree, how = 'left', on = ['hash_tab_num','date'])
merged_data = pd.merge(merged_data, kolvo_smen, how = 'left', on = ['hash_tab_num','date'])
merged_data = pd.merge(merged_data, kolvo_bolni4, how = 'left', on = ['hash_tab_num','date'])
merged_data = merged_data.drop_duplicates()

In [10]:
# Создание 12ти столбцов с датами будущих периодов для формирования таргетов

merged_data['sick'] = merged_data['sick'].fillna(0)
merged_data['target_dates'] = merged_data['date'].apply(lambda x: pd.date_range((x),\
    periods = 13, freq='1MS',closed = 'right'))
new_target_dates = pd.DataFrame(merged_data['target_dates'].tolist(), \
    columns = ['y_dt_'+str(i) for i in range(1,13)], index = merged_data.index)
merged_data = pd.merge(merged_data,new_target_dates, left_index=True, right_index=True)
merged_data.drop(['target_dates'],axis = 1, inplace = True)
merged_data['date'] = pd.to_datetime(merged_data['date'])

In [11]:
# Присоединение данных о больничных к будущим периодам созданным на предыдущем шаге

for i in range(1,13):
    dt_col_name = 'y_dt_'+str(i)
    y_col_name = 'y_'+str(i)
    
    targets_tmp = merged_data[['date','hash_tab_num','sick']]
    targets_tmp.columns = [dt_col_name, 'hash_tab_num', y_col_name]
    
    merged_data = pd.merge(merged_data, targets_tmp, how = 'left', on = [dt_col_name, 'hash_tab_num'])
    
    merged_data.drop(dt_col_name, axis = 1, inplace = True)

In [12]:
# Деление на трейн и тест для последующего прогноза

train = merged_data[merged_data.date < pd.to_datetime('2019-08-01')]
submission_extra = merged_data[merged_data.date == pd.to_datetime('2019-08-01')]
submission_extra.index = range(len(submission_extra))

In [13]:
# Обучение модели 12 раз с осуществлением предсказания на 12 месяцев.

submission_extra_pred = pd.DataFrame()
for i in range(1,13):
    y_col_name = 'y_' + str(i) 
    X = train.dropna(subset=[y_col_name])\
    .drop(['y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 
        'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12',
        'date', 'hash_tab_num'], axis = 1)
    
    X.fillna(0, inplace=True)
    
    y = train.dropna(subset=[y_col_name])[y_col_name]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, 
                                                        shuffle=True, stratify=y)

    X_submission_extra = submission_extra\
    .drop(['y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 
        'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12',
        'date', 'hash_tab_num'], axis = 1)

    X_submission_extra.fillna(0, inplace=True)

    model = RandomForestClassifier()

    model.fit(X_train, y_train)

    
    p, r, thresholds = precision_recall_curve(y_test, model.predict_proba(X_test)[:,1])
    f1_scores = 2*r*p / (r+p)
    f1_scores = f1_scores[p > 0]
    th = thresholds[np.argmax(f1_scores)]

    submission_extra_pred[y_col_name] = (model.predict_proba(X_submission_extra)[:,1] >= th).astype(int)

submission_extra_pred['hash_tab_num'] = submission_extra['hash_tab_num']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


In [14]:
# Представление результата работы модели в плоский вид

one_line_sub = pd.DataFrame(columns = ['hash_tab_num','date','target'])
for i in range(1,13):
    temp_result = pd.DataFrame(columns = ['hash_tab_num','date','target'])
    temp_result['hash_tab_num'] = submission_extra_pred['hash_tab_num']
    temp_result['date'] = pd.to_datetime('2019-09-01') + pd.DateOffset(months=i-1)
    temp_result['target'] = submission_extra_pred['y_' + str(i)]
    one_line_sub = pd.concat([one_line_sub, temp_result], axis = 0)

In [15]:
one_line_sub.head()

,hash_tab_num,date,target
0,0,2019-09-01,1
1,1,2019-09-01,0
2,2,2019-09-01,0
3,3,2019-09-01,0
4,4,2019-09-01,1


In [30]:
# Подготовка файла submission
submission = pd.read_csv('submission_check_.csv', sep = ';')
submission.date = pd.to_datetime(submission.date, format='%Y-%m-%d')
submission.drop('target', axis =1, inplace = True)
submission.head()

,hash_tab_num,date
0,0,2019-09-01
1,0,2019-10-01
2,0,2019-11-01
3,0,2019-12-01
4,0,2020-01-01


In [31]:
one_line_sub['hash_tab_num'] = pd.to_numeric(one_line_sub['hash_tab_num'])
submission_final = pd.merge(submission, one_line_sub, how = 'left', on = ['hash_tab_num','date'])


In [32]:
submission_final.head()

,hash_tab_num,date,target
0,0,2019-09-01,1
1,0,2019-10-01,1
2,0,2019-11-01,1
3,0,2019-12-01,1
4,0,2020-01-01,1


In [33]:
submission_final.to_csv('my_submission.csv', sep=';', index=False)